<a href="https://colab.research.google.com/github/iskra3138/git_tutorial/blob/master/Lime_Test_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
ls

In [0]:
!pip install lime

In [0]:
!git clone https://github.com/marcotcr/tf-models.git

In [0]:
!wget http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz

In [0]:
!tar -xvf inception_v3_2016_08_28.tar.gz

In [0]:
!wget https://www.researchgate.net/profile/Amin_Karbasi/publication/318460835/figure/fig2/AS:530639130955776@1503525635077/Original-image-and-visualization-of-the-superpixels-selected-by-the-three-algorithms-to.png

In [0]:
ls

In [0]:
!rm inception_v3_2016_08_28.tar.gz

In [0]:
import tensorflow as tf
slim = tf.contrib.slim
import sys
sys.path.append('./tf-models/slim')
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

Create a predict fn for inception v3, takes in a list of images and returns a matrix of prediction probabilities

In [0]:
from nets import inception
from preprocessing import inception_preprocessing

In [0]:
session = tf.Session()

In [0]:
image_size = inception.inception_v3.default_image_size
def transform_img_fn(path_list):
    out = []
    for f in path_list:
        image_raw = tf.image.decode_jpeg(open(f).read(), channels=3)
        image = inception_preprocessing.preprocess_image(image_raw, image_size, image_size, is_training=False)
        out.append(image)
    return session.run([out])[0]

In [0]:
from datasets import imagenet
names = imagenet.create_readable_names_for_imagenet_labels()

In [0]:
processed_images = tf.placeholder(tf.float32, shape=(None, 299, 299, 3))

In [0]:
import os
with slim.arg_scope(inception.inception_v3_arg_scope()):
    logits, _ = inception.inception_v3(processed_images, num_classes=1001, is_training=False)
probabilities = tf.nn.softmax(logits)

checkpoints_dir = './'
init_fn = slim.assign_from_checkpoint_fn(
    os.path.join(checkpoints_dir, 'inception_v3.ckpt'),
    slim.get_model_variables('InceptionV3'))
init_fn(session)

In [0]:
def predict_fn(images):
    return session.run(probabilities, feed_dict={processed_images: images})

In [0]:
'''from google.colab import files

uploaded = files.upload()'''

In [0]:

#images = transform_img_fn(['./dogs.png'])
images = transform_img_fn(['./Original-image-and-visualization-of-the-superpixels-selected-by-the-three-algorithms-to.png'])
# I'm dividing by 2 and adding 0.5 because of how this Inception represents images
plt.imshow(images[0] / 2 + 0.5)
preds = predict_fn(images)
for x in preds.argsort()[0][-5:]:
    print (x, names[x], preds[0,x])

In [0]:
image = images[0]

In [0]:
## Now let's get an explanation

In [0]:
from lime import lime_image
import time

In [0]:
explainer = lime_image.LimeImageExplainer()

In [0]:
tmp = time.time()
# Hide color is the color for a superpixel turned OFF. Alternatively, if it is NONE, the superpixel will be replaced by the average of its pixels
explanation = explainer.explain_instance(image, predict_fn, top_labels=5, hide_color=0, num_samples=1000)
print time.time() - tmp

Now let's see the explanation for the top class (Bernese mountain dog)

We can see the top 5 superpixels that are most positive towards the class with the rest of the image hidden

In [0]:
from skimage.segmentation import mark_boundaries

In [0]:
temp, mask = explanation.get_image_and_mask(240, positive_only=True, num_features=1, hide_rest=True)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

In [0]:
temp, mask = explanation.get_image_and_mask(240, positive_only=True, num_features=2, hide_rest=True)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

In [0]:
temp, mask = explanation.get_image_and_mask(240, positive_only=True, num_features=3, hide_rest=True)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

In [0]:
temp, mask = explanation.get_image_and_mask(240, positive_only=True, num_features=4, hide_rest=True)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

In [0]:
temp, mask = explanation.get_image_and_mask(240, positive_only=True, num_features=10, hide_rest=True)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

In [0]:
temp, mask = explanation.get_image_and_mask(240, positive_only=True, num_features=11, hide_rest=True)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

In [0]:
temp, mask = explanation.get_image_and_mask(240, positive_only=True, num_features=7, hide_rest=True)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

In [0]:
temp, mask = explanation.get_image_and_mask(240, positive_only=False, num_features=10000, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

In [0]:
temp, mask = explanation.get_image_and_mask(240, positive_only=False, num_features=100, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

In [0]:
temp, mask = explanation.get_image_and_mask(240, positive_only=True, num_features=5, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

In [0]:
temp, mask = explanation.get_image_and_mask(240, positive_only=False, num_features=10, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

In [0]:
temp, mask = explanation.get_image_and_mask(240, positive_only=False, num_features=1000, hide_rest=False, min_weight=0.1)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

In [0]:
temp, mask = explanation.get_image_and_mask(240, positive_only=False, num_features=1000, hide_rest=False, min_weight=0.1)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

Let's see the explanation for Egyptian cat

Most positive towards egyptian cat:

In [0]:
temp, mask = explanation.get_image_and_mask(223, positive_only=True, num_features=5, hide_rest=True)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

Pros and cons:

In [0]:
temp, mask = explanation.get_image_and_mask(223, positive_only=False, num_features=10, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))